In [1]:
#SMC DNA Meta Challange
#https://www.synapse.org/#!Synapse:syn4588939/wiki/233672

In [2]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

In [3]:
#Read the data

GenomicFeatures_CPCG0100 = pd.read_csv(open('data/CPCG0100/GenomicFeatures_CPCG0100.txt','r'),sep='\t')
SNVCalls_CPCG0100    = pd.read_csv(open('data/CPCG0100/SNVCalls_CPCG0100.txt','r'),sep='\t') 

In [4]:
print GenomicFeatures_CPCG0100.shape
print SNVCalls_CPCG0100.shape

(198526, 15)
(198526, 182)


In [5]:
np.unique(GenomicFeatures_CPCG0100['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CNN', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA',
       'GCC', 'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'NCG', 'NTC',
       'TCA', 'TCC', 'TCG', 'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [6]:
GenomicFeatures_CPCG0100[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10408,183,3,4490,231,133,25,55,TTA,4.696078,0.587065,NaN,41830,0.367965
1,chr1,10413,177,10,4691,229,134,25,60,CTA,4.528846,0.587065,NaN,41825,0.371179
2,chr1,16580,115,10,3512,137,74,0,52,TCA,2.707143,0.532338,NaN,35658,0.335766
3,chr1,17519,83,26,3251,119,59,37,38,GCA,2.746377,0.676617,ncRNA_intronic,34719,0.462185
4,chr1,17538,96,20,3255,127,53,29,47,GCC,2.719424,0.676617,NaN,34700,0.409449


In [7]:
SNVCalls_CPCG0100[:5]

,CHROM,POS,END,X2399002,X2673208,X2673225,X2677201,X2677205,X2679037,X2679040,X2711267,X2762064,X2762066,X2787011,X2787148,X2787658,X2787662,X2788241,X2788246,X2788251,
0,chr1,10407,10408,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,chr1,10412,10413,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,chr1,16579,16580,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,chr1,17518,17519,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
4,chr1,17537,17538,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [8]:
CPCG0100 = pd.merge(GenomicFeatures_CPCG0100, SNVCalls_CPCG0100,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [9]:
CPCG0100[:5]

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399002,X2673208,X2673225,
0,chr1,10408,183,3,4490,231,133,25,55,TTA,4.696078,0.587065,NaN,41830,0.367965,10407,10408,0,0,0,...
1,chr1,10413,177,10,4691,229,134,25,60,CTA,4.528846,0.587065,NaN,41825,0.371179,10412,10413,0,0,0,...
2,chr1,16580,115,10,3512,137,74,0,52,TCA,2.707143,0.532338,NaN,35658,0.335766,16579,16580,0,0,0,...
3,chr1,17519,83,26,3251,119,59,37,38,GCA,2.746377,0.676617,ncRNA_intronic,34719,0.462185,17518,17519,0,0,0,...
4,chr1,17538,96,20,3255,127,53,29,47,GCC,2.719424,0.676617,NaN,34700,0.409449,17537,17538,0,0,0,...


In [10]:
CPCG0100.shape #182+15

(198526, 196)

In [11]:
CPCG0100_X = CPCG0100.iloc[:,2:15] #Exclude CHROM, POS
CPCG0100_Y = CPCG0100.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [12]:
print CPCG0100_X.shape
print CPCG0100_Y.shape

(198526, 13)
(198526, 179)


In [13]:
CPCG0100_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,183,3,4490,231,133,25,55,TTA,4.696078,0.587065,NaN,41830,0.367965
1,177,10,4691,229,134,25,60,CTA,4.528846,0.587065,NaN,41825,0.371179


In [14]:
CPCG0100_Y[:2]

,X2399002,X2673208,X2673225,X2677201,X2677205,X2679037,X2679040,X2711267,X2762064,X2762066,X2787011,X2787148,X2787658,X2787662,X2788241,X2788246,X2788251,X2788254,X2791036,X2791292,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [15]:
totalSample = CPCG0100_Y.shape[0]
CPCG0100_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print CPCG0100_Y_Val.shape

198526
(198526, 1)


In [16]:
CPCG0100_Y.shape

(198526, 179)

In [17]:
CPCG0100_Y.values[0].shape

(179,)

In [18]:
for i in range(totalSample):
    CPCG0100_Y_Val[i] = np.argmax(np.bincount(CPCG0100_Y.values[i]))

In [19]:
CPCG0100_Y_Val_1 = pd.DataFrame(CPCG0100_Y_Val)
CPCG0100_Y_Val_1.columns = ['Label']

In [20]:
CPCG0100_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [21]:
CPCG0100_dataframe = pd.concat([CPCG0100_X, CPCG0100_Y_Val_1],axis=1)

In [22]:
CPCG0100_dataframe.shape

(198526, 14)

In [23]:
#CPCG0100_dataframe.to_csv('CPCG0100.csv')

In [24]:
#CPCG0100_dataframe.iloc[:20000,:].to_csv('CPCG0100_small.csv')

In [25]:
#GenomicFeatures_CPCG0100 = pd.read_csv('CPCG0100.csv')

In [26]:
#Now we have variable importance ...lets do analysis

In [27]:
var_importance = pd.read_csv(open('CPCG0100_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [28]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.332800
2,TumourCoverage,0.146610,0.146610,0.048792
3,NormalCoverage,0.133090,0.133090,0.044292
4,BaseQual,0.108282,0.108282,0.036036
5,RefAllele,0.090738,0.090738,0.030198


In [29]:
GenomicFeatures_CPCG0100[:5]


,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10408,183,3,4490,231,133,25,55,TTA,4.696078,0.587065,NaN,41830,0.367965
1,chr1,10413,177,10,4691,229,134,25,60,CTA,4.528846,0.587065,NaN,41825,0.371179
2,chr1,16580,115,10,3512,137,74,0,52,TCA,2.707143,0.532338,NaN,35658,0.335766
3,chr1,17519,83,26,3251,119,59,37,38,GCA,2.746377,0.676617,ncRNA_intronic,34719,0.462185
4,chr1,17538,96,20,3255,127,53,29,47,GCC,2.719424,0.676617,NaN,34700,0.409449


In [30]:
#Function that sums according to the ratio
GenomicFeatures_CPCG0100.columns

Index([u'CHROM', u'POS', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'DistanceSNP', u'StrandBias'], dtype='object')

In [31]:
var_importance['variable'].shape

(63,)

In [32]:
noOfSample = GenomicFeatures_CPCG0100.shape[0]

In [33]:
var_importance[:]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.332800
2,TumourCoverage,0.146610,0.146610,0.048792
3,NormalCoverage,0.133090,0.133090,0.044292
4,BaseQual,0.108282,0.108282,0.036036
5,RefAllele,0.090738,0.090738,0.030198
6,GenomicLocation.upstream,0.075580,0.075580,0.025153
7,Trinucleotide.GTG,0.061212,0.061212,0.020371
8,GenomicLocation.ncRNA_UTR5,0.059416,0.059416,0.019774
9,MapQual,0.056647,0.056647,0.018852
10,GenomicLocation.ncRNA_splicing,0.055621,0.055621,0.018511


In [34]:
var_importance.iloc[0,0]

'NonRefAllele'

In [35]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.108282387256622,
 'DistanceSNP': 0.0296818818897009,
 'GCcontent': 0.0099623901769518904,
 'GenomicLocation.UTR3': 0.040696121752262102,
 'GenomicLocation.UTR5': 0.049301221966743497,
 'GenomicLocation.downstream': 0.054298251867294298,
 'GenomicLocation.exonic': 0.0135910073295236,
 'GenomicLocation.exonic;splicing': 0.046713117510080303,
 'GenomicLocation.intergenic': 0.0079197343438863789,
 'GenomicLocation.intronic': 0.0058423839509487204,
 'GenomicLocation.missing(NA)': 0.0151335271075368,
 'GenomicLocation.ncRNA_UTR3': 0.050084210932254798,
 'GenomicLocation.ncRNA_UTR5': 0.059415847063064603,
 'GenomicLocation.ncRNA_exonic': 0.051896296441555002,
 'GenomicLocation.ncRNA_intronic': 0.021088972687721301,
 'GenomicLocation.ncRNA_splicing': 0.055620636790990802,
 'GenomicLocation.splicing': 0.054530497640371302,
 'GenomicLocation.upstream': 0.075579859316349002,
 'GenomicLocation.upstream;downstream': 0.038184694945812198,
 'HomopolymerRate': 0.050568535923957797,
 'Ma

In [36]:
var_imp_dict['RefAllele']

0.090738072991371196

In [37]:
var_imp_dict['Trinucleotide.'+GenomicFeatures_CPCG0100.iloc[0,9]]
print str(2.3)

2.3


In [38]:
GenomicFeatures_CPCG0100.iloc[0,12]

nan

In [39]:
CPCG0100_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_CPCG0100.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_CPCG0100.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_CPCG0100.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_CPCG0100.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_CPCG0100.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_CPCG0100.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_CPCG0100.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_CPCG0100.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_CPCG0100.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_CPCG0100.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_CPCG0100.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_CPCG0100.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_CPCG0100.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_CPCG0100.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_CPCG0100.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_CPCG0100.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' DistanceSNP=',DistanceSNP
    #print 'StrandBias=',StrandBias
    CPCG0100_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [40]:
CPCG0100_Dict[10408]

1801.1287921620042

In [41]:
#Sorting dictionary
CPCG0100_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in CPCG0100_Dict.items()],reverse=True)]

In [42]:
CPCG0100_Dict_Frame = pd.DataFrame(CPCG0100_Dict1, columns=['POS', 'Value'])

In [43]:
CPCG0100_Dict_Frame

,POS,Value
0,27732070,124356.210469
1,63653,78465.485536
2,185807,75245.760788
3,61969229,74876.764039
4,61969437,73075.770793
5,61968977,69954.301688
6,61969180,68723.270131
7,61969371,68071.385322
8,61968871,64905.798355
9,61969385,63511.022735


In [44]:
CPCG0100_Dict_Frame.to_csv("CPCG0100_Dict_Frame.csv")

In [128]:
#CPCG0100_Dict_Frame

In [45]:
CPCG0100_Dict_Frame[:10]

,POS,Value
0,27732070,124356.210469
1,63653,78465.485536
2,185807,75245.760788
3,61969229,74876.764039
4,61969437,73075.770793
5,61968977,69954.301688
6,61969180,68723.270131
7,61969371,68071.385322
8,61968871,64905.798355
9,61969385,63511.022735


In [57]:
rows = CPCG0100_Dict_Frame['POS'][:10].values
print rows
GenomicFeatures_CPCG0100_Top10 = GenomicFeatures_CPCG0100.loc[GenomicFeatures_CPCG0100['POS'].isin(rows)]

[27732070    63653   185807 61969229 61969437 61968977 61969180 61969371
 61968871 61969385]


In [58]:
GenomicFeatures_CPCG0100_Top10

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
80227,chr19,27732070,6965,1035,249892,357720,322540,17,65,CTT,2.342466,0.383085,NaN,8640,0.540655
160236,chr7,61968871,6982,899,236654,138975,126878,37,42,ATT,2.798561,0.358209,NaN,16301,0.604871
160237,chr7,61968977,7143,853,244685,153132,142931,60,53,ATG,2.178808,0.402985,NaN,16195,0.521544
160238,chr7,61969180,7436,644,224614,157156,147005,36,44,TTT,2.791367,0.353234,NaN,15992,0.520750
160239,chr7,61969229,7162,829,247015,170379,159252,46,49,CCT,2.545455,0.378109,NaN,15943,0.540501
160240,chr7,61969371,7660,374,240318,148311,140982,60,45,ACT,2.517241,0.417910,NaN,15801,0.499740
160241,chr7,61969385,7025,437,201465,147666,139001,60,43,TTT,2.591549,0.417910,NaN,15787,0.514465
160243,chr7,61969437,7810,185,251866,163187,154133,60,51,GCA,2.573427,0.398010,NaN,15735,0.475877
185771,chrX,63653,1,0,24,5,2,0,27,ACC,4.327731,0.716418,intergenic,2643407,0.000000
185773,chrX,185807,113,4,3354,149,77,0,53,ATG,3.533333,0.601990,intergenic,2521253,0.469799
